In [5]:
from datasets.labelers import Labeler
import pandas as pd
import os

In [3]:
cohort_path = '/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/cohort'

In [2]:
def read_file(filename, columns=None, **kwargs):
    '''
    Helper function to read parquet and csv files into DataFrame
    '''
    print(filename)
    load_extension = os.path.splitext(filename)[-1]
    if load_extension == ".parquet":
        return pd.read_parquet(filename, columns=columns,**kwargs)
    elif load_extension == ".csv":
        return pd.read_csv(filename, usecols=columns, **kwargs)

In [53]:
cohort = read_file(
            os.path.join(
                cohort_path,
                "cohort_split_no_nb.parquet"
            ),
            engine='pyarrow'
        )

/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/cohort/cohort_split_no_nb.parquet


In [7]:
labeler = Labeler()

/local-scratch/nigam/envs/jlemmon/conl/lib/python3.9/site-packages/google/auth/_default.py:81: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [8]:

labeler.configure(
    rs_dataset="jlemmon_explore",
    cohort_name="tl_admission_rollup_temp",
    target_table_name="lab_test_admissions_rollup_filtered_labeled",
)

In [31]:

# obtain all labels except for readmission
labeler.create_label_table(exclude_labeler_ids=['mortality','los_7','icu_admission','readmission_30'])

df = pd.read_gbq(
    "select * from `som-nero-nigam-starr.jlemmon_explore.lab_test_admissions_rollup_filtered_labeled`",
    use_bqstorage_api=True
)

,person_id,admit_date,discharge_date,age_days,pediatric_age_group,age_group,sex,race,hyperkalemia_lab_max_potassium,hyperkalemia_lab_mild_measurement,...,anemia_dx_label,anemia_dx_start_datetime,hyperkalemia_dx_label,hyperkalemia_dx_start_datetime,hyponatremia_dx_label,hyponatremia_dx_start_datetime,thrombocytopenia_dx_label,thrombocytopenia_dx_start_datetime,neutropenia_dx_label,neutropenia_dx_start_datetime
0,30055184,2016-11-19 10:40:00,2016-11-20 12:08:00,18573,non-pediatric,"[50,60)",FEMALE,White,NaN,NaN,...,0,NaT,0,NaT,0,NaT,0,NaT,0,NaT
1,30208448,2020-02-01 10:23:00,2020-02-02 11:05:00,7470,late adolescence,"[18,30)",MALE,White,NaN,NaN,...,0,NaT,0,NaT,0,NaT,0,NaT,0,NaT
2,30242723,2017-06-17 08:38:00,2017-06-19 12:05:00,20732,non-pediatric,"[50,60)",FEMALE,Other,NaN,NaN,...,0,NaT,0,NaT,0,NaT,0,NaT,0,NaT
3,30739499,2016-12-11 13:16:00,2016-12-13 12:27:00,0,term neonatal,"[0,18)",FEMALE,White,NaN,NaN,...,0,NaT,0,NaT,0,NaT,0,NaT,0,NaT
4,30777818,2014-07-19 18:40:00,2014-07-20 10:55:00,4,term neonatal,"[0,18)",FEMALE,Other,NaN,NaN,...,1,2014-07-19 18:40:00,0,NaT,0,NaT,0,NaT,0,NaT


In [72]:
merge_df = cohort.merge(df, how='left', on=['person_id', 'admit_date','discharge_date'])

In [74]:
merge_df.to_parquet(
		os.path.join(cohort_path, "cohort_no_nb.parquet"), engine="pyarrow", index=False,
	)